In [1]:
from AC import preproc
from AC import get_news
import inspect
import nltk
nltk.download("stopwords")

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
economia = get_news("./articoli_economia/")
cultura = get_news("./articoli_cultura/")
tech = get_news("./articoli_tech/")
politica = get_news("./articoli_politica/")
sport = get_news("./articoli_sport/")
cronaca = get_news("./articoli_cronaca/")

for articolo in economia:
    articolo['categoria'] = "Economia"
for articolo in cultura:
    articolo['categoria'] = "Cultura"
for articolo in tech:
    articolo['categoria'] = "Tech"
for articolo in politica:
    articolo['categoria'] = "Politica"
for articolo in sport:
    articolo['categoria'] = "Sport"
for articolo in cronaca:
    articolo['categoria'] = "Cronaca"
dati_preprocessati =  preproc(tech + politica + cultura + economia + sport + cronaca)

100%|██████████| 2400/2400 [03:43<00:00, 10.75it/s]


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import tqdm

In [4]:
seed = 0
train_documents,test_documents = train_test_split(dati_preprocessati,random_state=seed,train_size=0.5)
test_documents, val_documents = train_test_split(test_documents,random_state=seed,train_size=0.5)

/home/alex/anaconda3/envs/msbd/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
docs_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in dati_preprocessati]
train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]
val_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in val_documents]

docs_cats = [x["categoria"] for x in dati_preprocessati]
train_cats = [x["categoria"] for x in train_documents]
test_cats = [x["categoria"] for x in test_documents]
val_cats = [x["categoria"] for x in val_documents]

In [6]:
ldac = Pipeline([
    ("count_mx",CountVectorizer(max_features=1000000, lowercase=True)),
    ("lda", LatentDirichletAllocation(max_iter=50, learning_method='online',random_state=0)),
    ("classifier",DecisionTreeClassifier(random_state=0))
])
params = {
    'lda__n_components': 12,
    'lda__learning_decay': 0.7, 
    'count_mx__ngram_range': (1, 3), 
    'count_mx__min_df': 9,
    'count_mx__max_df': 400
}
ldac.set_params(**params)

Pipeline(memory=None,
     steps=[('count_mx', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=400, max_features=1000000, min_df=9,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
      ...         min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'))])

In [7]:
ldac.fit(train_texts, train_cats)

Pipeline(memory=None,
     steps=[('count_mx', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=400, max_features=1000000, min_df=9,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
      ...         min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'))])

In [ ]:
# nuovi articoli (possibilmente non gia' presenti nel dataset) da classificare

In [8]:
from estrattore_dati.estrattore_articoli_ansa import estrai
p = estrai("http://www.ansa.it/sito/notizie/tecnologia/hitech/2019/06/03/amazon-apre-negozio-a-manchester_24353478-6d22-43b4-89d6-32725e95bf09.html")
#p = estrai("http://www.ansa.it/sito/notizie/politica/2019/06/09/bufera-procure-ora-lanm-rischia-la-crisi_03006b0c-effb-4b43-be1b-1cf40ab78c2b.html")
#p = estrai("http://www.ansa.it/sito/notizie/sport/tennis/2019/06/10/woods-e-molinari-celebrano-nadal_e7e5fb39-7c09-45d1-be35-65691117c3d2.html")
#p = estrai("http://www.ansa.it/sito/notizie/cultura/musica/2019/06/09/ligabue-niente-e-come-un-concerto_1948cb23-6b91-4eec-92eb-d5e255556de1.html")
#p = estrai("http://www.ansa.it/sito/notizie/tecnologia/hitech/2019/06/13/uber-testa-drone-per-i-pasti-a-domicilio_24010da2-972b-4d3b-bfeb-ea62f584111d.html")
#p = estrai("http://www.ansa.it/sito/notizie/politica/2019/06/11/governo-conte-di-maio-salvini-ue_0eeb81e1-d058-480b-8550-9b9524a35060.html")
#p = estrai("http://www.ansa.it/sito/notizie/sport/calcio/2019/06/13/il-presidente-della-lega-pro-ghirelli-ospite-del-forum-ansa_1646e537-9880-4e92-b64e-f45281d7cf69.html")
#p = estrai("http://www.ansa.it/sito/notizie/cronaca/2019/06/12/dl-sicurezza-bis-da-multe-a-navi-a-fondo-rimpatri-_e93de1be-fffb-4f92-a2b1-a52ed246a6fe.html")
#p = estrai("http://www.ansa.it/sito/notizie/cronaca/2019/06/12/addio-ai-furbetti-del-cartellino-arrivano-i-controlli-biometrici_eb36c913-b809-4ae0-b59c-5087222d8c60.html")
#p = estrai("http://www.ansa.it/sito/notizie/tecnologia/software_app/2019/06/12/facebook-paga-utenti-per-dati-su-uso-app_fee73f87-6558-486f-9ec5-be509c3cacef.html")
x_1 = preproc([p])

ldac.predict([' '.join(x_1[0]['testo'] +x_1[0]['titolo_articolo']+x_1[0]['sottotitolo'] + x_1[0]['tags'])])

100%|██████████| 1/1 [00:00<00:00, 11.96it/s]


array(['Tech'], dtype='<U8')